In [13]:
import tensorflow as tf
import keras as ks
import pandas as pd
from imblearn.under_sampling import NearMiss
from sklearn.model_selection import train_test_split
import numpy as np

# Загрузка датасетов и предобработка

In [14]:
data_class = pd.read_csv('../data/rainAUS_prepared.csv')
data_class

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,Cloud9am,Cloud3pm,Temp9am,Temp3pm,ord__Location,ord__WindDir9am,ord__WindGustDir,ord__WindDir3pm,ord__RainToday,RainTomorrow
0,12,13.4,22.9,0.6,4.0,44.0,20.0,24.0,71.0,22.0,...,8.0,7.0,16.9,21.8,2.0,13.0,13.0,14.0,0.0,0
1,12,7.4,25.1,0.0,4.0,44.0,4.0,22.0,44.0,25.0,...,7.0,7.0,17.2,24.3,2.0,6.0,14.0,15.0,0.0,0
2,12,12.9,25.7,0.0,4.0,46.0,19.0,26.0,38.0,30.0,...,7.0,2.0,21.0,23.2,2.0,13.0,15.0,15.0,0.0,0
3,12,9.2,28.0,0.0,4.0,24.0,11.0,9.0,45.0,16.0,...,7.0,7.0,18.1,26.5,2.0,9.0,4.0,0.0,0.0,0
4,12,17.5,32.3,1.0,4.0,41.0,7.0,20.0,82.0,33.0,...,7.0,8.0,17.8,29.7,2.0,1.0,13.0,7.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142188,6,3.5,21.8,0.0,4.0,31.0,15.0,13.0,59.0,27.0,...,7.0,7.0,9.4,20.9,41.0,2.0,0.0,0.0,0.0,0
142189,6,2.8,23.4,0.0,4.0,31.0,13.0,11.0,51.0,24.0,...,7.0,7.0,10.1,22.4,41.0,9.0,0.0,1.0,0.0,0
142190,6,3.6,25.3,0.0,4.0,22.0,13.0,9.0,56.0,21.0,...,7.0,7.0,10.9,24.5,41.0,9.0,6.0,3.0,0.0,0
142191,6,5.4,26.9,0.0,4.0,37.0,9.0,9.0,53.0,24.0,...,7.0,7.0,12.5,26.1,41.0,9.0,3.0,14.0,0.0,0


In [15]:
data_reg_white = pd.read_csv('../data/winequality-white.csv', sep = ';')
data_reg_red = pd.read_csv('../data/winequality-red.csv', sep = ';')
data_reg_white['color'] = 0
data_reg_red['color'] = 1
data_reg = pd.concat([data_reg_white, data_reg_red], axis = 0)
data_reg.reset_index()
data_reg

data_reg.to_csv('../data/winequality', index= False)

# Разделение выборок

In [16]:
y_cl = data_class['RainTomorrow'].to_numpy()
X_cl = data_class.drop(['RainTomorrow'], axis=1).to_numpy()

In [17]:
nm = NearMiss()
X_train_cl, X_test_cl, y_train_cl, y_test_cl = train_test_split(X_cl, y_cl, test_size=0.2)
X_train_cl, y_train_cl = nm.fit_resample(X_train_cl, y_train_cl.ravel())

In [18]:
y_reg = data_reg['quality'].to_numpy()
X_reg = data_reg.drop(['quality'], axis=1).to_numpy()
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, stratify = y_reg)

# Построение НС для регрессии

In [19]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score

In [20]:
reg_stats = lambda y, y_pred: [
    print(f'mape : {mean_absolute_percentage_error(y, y_pred)}'), 
    print(f'r2: {r2_score(y, y_pred)}')
]

In [21]:
model_regression = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(12,)),
        tf.keras.layers.Dense(32, activation="linear"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(1, activation="linear"),
    ]
)

In [22]:
model_regression.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="mse")

In [23]:
model_regression.fit(X_train_reg, y_train_reg, epochs=500)

Epoch 1/500
163/163 [==============================] - 1s 690us/step - loss: 19.1530
Epoch 2/500
163/163 [==============================] - 0s 708us/step - loss: 2.0556
Epoch 3/500
163/163 [==============================] - 0s 671us/step - loss: 1.8977
Epoch 4/500
163/163 [==============================] - 0s 677us/step - loss: 1.8699
Epoch 5/500
163/163 [==============================] - 0s 671us/step - loss: 1.7703
Epoch 6/500
163/163 [==============================] - 0s 659us/step - loss: 1.5880
Epoch 7/500
163/163 [==============================] - 0s 696us/step - loss: 1.5533
Epoch 8/500
163/163 [==============================] - 0s 659us/step - loss: 1.4740
Epoch 9/500
163/163 [==============================] - 0s 671us/step - loss: 1.3773
Epoch 10/500
163/163 [==============================] - 0s 671us/step - loss: 1.3231
Epoch 11/500
163/163 [==============================] - 0s 659us/step - loss: 1.2716
Epoch 12/500
163/163 [==============================] - 0s 690us/step - l

In [24]:
reg_stats(y_test_reg, model_regression.predict(X_test_reg))

41/41 [==============================] - 0s 524us/step
mape : 0.09959947179961992
r2: 0.34623674222426715


[None, None]

# Построение НС для классификации

In [60]:
from sklearn.metrics import f1_score, classification_report

In [61]:
class_stats = lambda y, y_pred: [
    print(classification_report(y, y_pred)),
    print(f'f1 : {f1_score(y, y_pred)}') 
]

In [63]:
data_class.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 22 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Date              142193 non-null  int64  
 1   MinTemp           142193 non-null  float64
 2   MaxTemp           142193 non-null  float64
 3   Rainfall          142193 non-null  float64
 4   Evaporation       142193 non-null  float64
 5   WindGustSpeed     142193 non-null  float64
 6   WindSpeed9am      142193 non-null  float64
 7   WindSpeed3pm      142193 non-null  float64
 8   Humidity9am       142193 non-null  float64
 9   Humidity3pm       142193 non-null  float64
 10  Pressure9am       142193 non-null  float64
 11  Pressure3pm       142193 non-null  float64
 12  Cloud9am          142193 non-null  float64
 13  Cloud3pm          142193 non-null  float64
 14  Temp9am           142193 non-null  float64
 15  Temp3pm           142193 non-null  float64
 16  ord__Location     14

In [89]:
model_classification_1 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(32, activation="relu", input_shape=(21,)),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)

In [90]:
model_classification_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="binary_crossentropy")

In [91]:
model_classification_1.fit(X_train_cl, y_train_cl, epochs=100, verbose=None)

In [92]:
y_pred = np.around(model_classification_1.predict(X_test_cl, verbose=None))
class_stats(y_test_cl, y_pred)

              precision    recall  f1-score   support

           0       0.88      0.75      0.81     22092
           1       0.42      0.63      0.50      6347

    accuracy                           0.72     28439
   macro avg       0.65      0.69      0.66     28439
weighted avg       0.77      0.72      0.74     28439

f1 : 0.5033679571923199


[None, None]